In [2]:
import io
import keras
import tensorflow as tf
import os
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#test_df=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/test_outs.txt',delimiter=",",header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
file = open('drive/My Drive/Colab Notebooks/dataset/vocab.txt', 'r')
# read all text
vocab = file.readlines()
vocabulary={}
for w in vocab:
  vocabulary[w.strip()]=""
# close the file
file.close()

In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:
docs_filtered=[]
for d in docs:
  # split into tokens by white space
  tokens = d.split()
  temp=""
  for t in tokens:
    if t in vocabulary:
      temp=temp+" "+t
  docs_filtered.append(temp)


In [0]:
# integer encode the documents
from keras.preprocessing.text import one_hot
vocab_size = len(vocabulary)+1

encoded_docs = [one_hot(d, vocab_size) for d in docs_filtered]

In [7]:
# pad documents to a max length of 4 words
max_length = 200
padded_docs = keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

91795


In [8]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape)


(91795, 8)


In [9]:
emb_dim=32
emb_model = tf.keras.Sequential([ 
    tf.keras.layers.Input(shape=(max_length,)) ,                           
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.GRU(128,return_sequences=True,dropout=0.3),
    tf.keras.layers.GRU(64,return_sequences=True,dropout=0.3),
    tf.keras.layers.GRU(32,return_sequences=True,dropout=0.3),
    tf.keras.layers.GRU(16),
    tf.keras.layers.Dense(8, activation="softmax")
])

emb_model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['acc'])
print(emb_model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           8000032   
_________________________________________________________________
gru (GRU)                    (None, 200, 128)          62208     
_________________________________________________________________
gru_1 (GRU)                  (None, 200, 64)           37248     
_________________________________________________________________
gru_2 (GRU)                  (None, 200, 32)           9408      
_________________________________________________________________
gru_3 (GRU)                  (None, 16)                2400      
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
Total params: 8,111,432
Trainable params: 8,111,432
Non-trainable params: 0
______________________________________________

In [10]:
emb_model.fit(padded_docs, dummy_y,epochs=30,batch_size=256, verbose=1,validation_split=0.1)


Epoch 1/30
323/323 [==============================] - 81s 250ms/step - loss: 1.8698 - acc: 0.2477 - val_loss: 1.6942 - val_acc: 0.2768
Epoch 2/30
323/323 [==============================] - 73s 227ms/step - loss: 1.1122 - acc: 0.5947 - val_loss: 0.6205 - val_acc: 0.8264
Epoch 3/30
323/323 [==============================] - 73s 225ms/step - loss: 0.4441 - acc: 0.8769 - val_loss: 0.3746 - val_acc: 0.8930
Epoch 4/30
323/323 [==============================] - 71s 220ms/step - loss: 0.2319 - acc: 0.9398 - val_loss: 0.3031 - val_acc: 0.9127
Epoch 5/30
323/323 [==============================] - 78s 241ms/step - loss: 0.1456 - acc: 0.9634 - val_loss: 0.3050 - val_acc: 0.9185
Epoch 6/30
323/323 [==============================] - 76s 234ms/step - loss: 0.1002 - acc: 0.9765 - val_loss: 0.3095 - val_acc: 0.9179
Epoch 7/30
323/323 [==============================] - 74s 228ms/step - loss: 0.0778 - acc: 0.9820 - val_loss: 0.3014 - val_acc: 0.9216
Epoch 8/30
323/323 [==============================] - 7

In [11]:
emb_model1 = tf.keras.Model(inputs=emb_model.inputs , outputs=emb_model.layers[3].output)
emb_X = emb_model1.predict(padded_docs)
print(emb_X.shape)
emb_X=np.reshape(emb_X, (emb_X.shape[0],1,-1))

(91795, 200, 32)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, dummy_y, test_size=0.18)

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(75271, 1, 6400)
(75271, 8)
(16524, 1, 6400)
(16524, 8)


In [17]:
model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.GRU(2048,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True,dropout=0.4,recurrent_dropout=0.3),
    tf.keras.layers.GRU(1024,return_sequences=True,dropout=0.4,recurrent_dropout=0.3),
    tf.keras.layers.GRU(512,return_sequences=True,dropout=0.3,recurrent_dropout=0.3),
    tf.keras.layers.GRU(256,return_sequences=True,dropout=0.3,recurrent_dropout=0.3),
    tf.keras.layers.GRU(64,return_sequences=True),
    tf.keras.layers.GRU(32),
    tf.keras.layers.Dense(8, activation="softmax")
])

In [18]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_26 (GRU)                 (None, 1, 2048)           51916800  
_________________________________________________________________
gru_27 (GRU)                 (None, 1, 1024)           9443328   
_________________________________________________________________
gru_28 (GRU)                 (None, 1, 512)            2362368   
_________________________________________________________________
gru_29 (GRU)                 (None, 1, 256)            591360    
_________________________________________________________________
gru_30 (GRU)                 (None, 1, 64)             61824     
_________________________________________________________________
gru_31 (GRU)                 (None, 32)                9408      
_________________________________________________________________
dense_4 (Dense)              (None, 8)                

In [19]:
history = model.fit(X_train, y_train, epochs=25,batch_size=256, shuffle=False, verbose=1,validation_split=0.1)
model.save('saved_model/text_classifier_rnn_word_embedding_epoch_20')

Epoch 1/25
265/265 [==============================] - 28s 106ms/step - loss: 0.1654 - acc: 0.9801 - val_loss: 0.0637 - val_acc: 0.9875
Epoch 2/25
265/265 [==============================] - 26s 100ms/step - loss: 0.0585 - acc: 0.9880 - val_loss: 0.0465 - val_acc: 0.9904
Epoch 3/25
265/265 [==============================] - 26s 99ms/step - loss: 0.0496 - acc: 0.9893 - val_loss: 0.0450 - val_acc: 0.9911
Epoch 4/25
265/265 [==============================] - 26s 100ms/step - loss: 0.0483 - acc: 0.9895 - val_loss: 0.0477 - val_acc: 0.9895
Epoch 5/25
265/265 [==============================] - 26s 99ms/step - loss: 0.0449 - acc: 0.9900 - val_loss: 0.0465 - val_acc: 0.9907
Epoch 6/25
265/265 [==============================] - 26s 100ms/step - loss: 0.0453 - acc: 0.9906 - val_loss: 0.0475 - val_acc: 0.9914
Epoch 7/25
265/265 [==============================] - 26s 99ms/step - loss: 0.0413 - acc: 0.9911 - val_loss: 0.0448 - val_acc: 0.9902
Epoch 8/25
265/265 [==============================] - 27s 

In [0]:
!ls saved_model

text_classifier_rnn_word_embedding_epoch_15
text_classifier_rnn_word_embedding_epoch_20


In [0]:
#model.load_weights('./checkpoints/text_classifier_rnn_word_embedding')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_word_embedding_epoch_20')

In [20]:
results = model.evaluate(X_test, y_test)
print(results)

517/517 [==============================] - 10s 18ms/step - loss: 0.0427 - acc: 0.9915
[0.04274992272257805, 0.9914669394493103]


In [0]:
y_pred=model.predict(X_test)

In [22]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

(8, 8)


In [23]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])

print(np.average(prec))
print(np.average(rec))
print(np.average(f_score))

99.1236616265443
99.09725740630013
99.10839164012037
